In [125]:
# TODO List

# solve the problem through a simple loop 
# solve the problem by using a shitty generator methods 
# solve the problem using functools 
# solve the problem using async.io 

# What is this tutorial about? 

I'm going to walk through some ways to use iPython widgets to tag data. I found it surprisingly hard to learn how to program up the control flow, so this tutorial walks through some terrible code that I wrote that didn't work, and ends up with some code that might get there in the end. 

# Problem definition 

The reason I got interested in this problem is that I want to speed up tagging of my montly spending, so for example if my bank statement has a report that looks like `costa - 3.95` I want to tag that as coffee, and a report like `sainsburies - 34.54` I will want to tag as groceries. 

I also want to bunch spending together and tag bunches of spends to make things even faster. I'll use this as the demo problem for this walkthrough, but the problem is generic. 

I have n items in a list

`["spend_1", "spend_2", "spend_3", ... "spend_n"]`

and I want to apply one tag from a set of tags to each item in the list

`["tag_1", "tag_2", "tag_3"]`

I have one additional tweak to this basic setup. I have grouped my spending into buckets, and If all of the items in the same bucket are of the same kind then I want to apply the tag to all of them at once, but if items in one bucket are wrong, then I want to break open the bucket and tag each item individually within that bucket. 

So my test data might look like this:

`
spends = [
    ["costa - 3.95", "costa - 4.95", "starbucks - 2.95"],
    ["sainsburies - 34.95", "sainsburies - 12.95", "tesco - 24.95"],
    ["BT - 34.95", "BT - 12.95", "BT - 24.95"],
    ["sainsburies - 34.95", "sainsburies - 12.95", "tesco - 24.95"],
    ["iTunes - 2.95", "iTunes - 3.95", "Amazon - 5.95"],
    ["costa - 3.95", "iTunes - 2.95", "costa - 4.95", "starbucks - 2.95", "costa - 3.95"]
]
`

In the above example the I've constructed the last bucket to have a bad grouping.

I might have the following tags for these kinds of buckets

`["coffee", "groceries", "bills", "movies"]`

# Quick introduction to displaying widgets in iPython. 

We want to display our results, and also display iPython buttons, using the widget framework from iPython, so that we can show the data that we want to tag along with buttons for tagging that data.

In [126]:
spends = [
    ["costa - 3.95", "costa - 4.95", "starbucks - 2.95"],
    ["sainsburies - 34.95", "sainsburies - 12.95", "tesco - 24.95"],
    ["BT - 34.95", "BT - 12.95", "BT - 24.95"],
    ["sainsburies - 34.95", "sainsburies - 12.95", "tesco - 24.95"],
    ["iTunes - 2.95", "iTunes - 3.95", "Amazon - 5.95"],
    ["costa - 3.95", "iTunes - 2.95", "costa - 4.95", "starbucks - 2.95", "costa - 3.95"]
]

tags = ["coffee", "groceries", "bills", "movies"]

## simple display area using widget.Output

In [127]:
from ipywidgets import interactive
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import widgets, Layout, Button

In [128]:
# create the display area
display_area = widgets.Output()
info_pane = widgets.VBox([display_area],layout=Layout(border="2px solid grey"))
display(info_pane)

In [130]:
# print and clear items from the display area
with display_area:
    for spend in spends:
#         clear_output()
        print(spend)

## Creating and displaying a button and a display area

In [131]:
button=Button(description="wow")
# create the display area
display_area = widgets.Output()
info_pane = widgets.VBox([display_area, button],layout=Layout(border="2px solid grey"))
display(info_pane)

In [132]:
# print and clear items from the display area
with display_area:
    for spend in spends:
        clear_output()
        print(spend)

## Creating and displaying a set of dynamically generated buttons and a display area

In [134]:
def set_buttons(button_labels):
    buttons = []
    for index, opt in  enumerate(button_labels):
        buttons.append(widgets.Button(description = opt))
    return buttons

In [135]:
buttons = set_buttons(tags)
display_area = widgets.Output()
layout = [] 
layout.append(display_area)
layout.extend(buttons)
info_pane = widgets.VBox(layout,layout=Layout(border="2px solid grey"))
display(info_pane)

In [136]:
# print and clear items from the display area
with display_area:
    for spend in spends:
        clear_output()
        print(spend)

## now bind out buttons on_click method to a function 

In [137]:
def set_buttons(button_labels, func):
    buttons = []
    for index, opt in  enumerate(button_labels):
        buttons.append(widgets.Button(description = opt))
        buttons[index].on_click(func) # set the binding function here 
    return buttons

def am_clicked(button):
    print(button.description)
    
buttons = set_buttons(tags, am_clicked)

display_area = widgets.Output()
layout = [] 
layout.append(display_area)
layout.extend(buttons)
info_pane = widgets.VBox(layout,layout=Layout(border="2px solid grey"))
display(info_pane)

coffee
groceries
bills
groceries
coffee
groceries
coffee
coffee
groceries
bills
movies


In [57]:
with display_area:
    for spend in spends:
        clear_output()
        print(spend)

# Soluiton 1 -- using a simple loop

The pattern we want is to run through our list of input, and on each item click a button and capture the button click associated with each item. Let's try to do this in a simple loop. 

In [58]:
display_area_one = widgets.Output()
layout = [] 
layout.append(display_area_one)
layout.extend(buttons)
info_pane = widgets.VBox(layout,layout=Layout(border="2px solid grey"))
display(info_pane)

In [138]:
with display_area_one:
    for spend in spends:
        # at this point I need to capture by button click before showing the next item. 
        # I need to force some kind of waiting state on this for loop. 
        # I don't know how to do this
        # there is no connection between the loop and the on_click method of the button :( 
        # 
        # it might be possible to dynamically bind the on_click method of the btton, and refresh the display area
        # here, but I don't know how to think about doing that.
        print(spend)

# Soluition 2 - use a generator to make it look like we are creating a callback into the on_click method

## Generators! 

In [140]:
for spend in spends:
    print(spend)

['costa - 3.95', 'costa - 4.95', 'starbucks - 2.95']
['sainsburies - 34.95', 'sainsburies - 12.95', 'tesco - 24.95']
['BT - 34.95', 'BT - 12.95', 'BT - 24.95']
['sainsburies - 34.95', 'sainsburies - 12.95', 'tesco - 24.95']
['iTunes - 2.95', 'iTunes - 3.95', 'Amazon - 5.95']
['costa - 3.95', 'iTunes - 2.95', 'costa - 4.95', 'starbucks - 2.95', 'costa - 3.95']


In [141]:
# we can create an iterator from a list like so
spend_iterator = iter(spends)

In [142]:
def show_spend(spend_iterator):
    spend = next(spend_iterator)
    print(spend)
    
# now each time we call `show_spend` we will see one item from the spends list
show_spend(spend_iterator)

['costa - 3.95', 'costa - 4.95', 'starbucks - 2.95']


In [143]:
show_spend(spend_iterator)

['sainsburies - 34.95', 'sainsburies - 12.95', 'tesco - 24.95']


## Our solution using generators 

how might this be useful? 
let's setup our display area again 

In [144]:
# create my iterator and hard code this specific iterator as an argument to a show progress function

spend_iterator = iter(spends)
def get_progress(button):
    """
    wraps the funciton bound to the on_click method
    """
    show_progress(button, spend_iterator, spends) # hard coding, bad, boo, bad ian. 
    
tagged_data = []
def show_progress(button, list_iterator, info_list):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, get_progress)) 
    # the buttons are bound to the get_progress function
    # this basically brings us back into this function, but one step after where we were
    with display_area_two: 
        clear_output()
        print(message) 
        display(buttons) 
        # we have a "button" context which was the button that was clicked in the last view
        current_index = info_list.index(current_value) # use this to find the location in the iter list
        tagged_data = info_list[current_index -1] # look back one to see what we tagged!!
        print(tagged_data)
        print("tagged as: " + button.description)

In [145]:
button = widgets.Button(description = "start") 
# as we don't have a loop we need to find a way to get into our iterator
# so we create this button to allow us to do that 

button.on_click(get_progress)
display(button)

display_area_two = widgets.Output()
info_pane = widgets.VBox([display_area_two],layout=Layout(border="2px solid grey"))
display(info_pane)

Button(description='start', style=ButtonStyle())

## Now let's add a complication, some default buttons! 

In [66]:
# create my iterator and hard code this specific iterator as an argument to a show progress function
spend_iterator = iter(spends)
def get_progress(button):
    show_progress(button, spend_iterator, spends) # hard coding, bad, boo, bad ian. 
    
tagged_data = []
def show_progress(button, list_iterator, info_list):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, get_progress)) 
    default_buttons = widgets.HBox(set_buttons(["stop", "split"], get_progress)) 
    # the buttons are bound to the get_progress function
    # this basically brings us back into this function, but one step after where we were
    with display_area_two: 
        clear_output()
        print(message) 
        display(buttons) 
        # we have a "button" context which was the button that was clicked in the last view
        current_index = info_list.index(current_value) # use this to find the location in the iter list
        tagged_data = info_list[current_index -1] # look back one to see what we tagged!!
        print(tagged_data)
        print("tagged as: " + button.description)
        display(default_buttons) 

In [69]:
button = widgets.Button(description = "start")
button.on_click(get_progress)
display(button)

progress_info = widgets.Output()
info_pane = widgets.VBox([display_area_two],layout=Layout(border="2px solid grey"))
display(info_pane)

Button(description='start', style=ButtonStyle())

## using the default buttons to try to control flow 

In [70]:
# create my iterator and hard code this specific iterator as an argument to a show progress function
spend_iterator = iter(spends)
def get_progress(button):
    show_progress(button, spend_iterator, spends) # hard coding, bad, boo, bad ian. 
    
tagged_data = []
def show_progress(button, list_iterator, info_list):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, get_progress)) 
    default_buttons = widgets.HBox(set_buttons(["stop", "split"], get_progress)) 
    # the buttons are bound to the get_progress function
    # this basically brings us back into this function, but one step after where we were
    with display_area_two: 
        if button.description == "split":
            clear_output()
            print("should try to split")
        else:
            clear_output()
            print(message) 
            display(buttons) 
            # we have a "button" context which was the button that was clicked in the last view
            current_index = info_list.index(current_value) # use this to find the location in the iter list
            tagged_data = info_list[current_index -1] # look back one to see what we tagged!!
            print(tagged_data)
            print("tagged as: " + button.description)
            display(default_buttons) 

In [71]:
button = widgets.Button(description = "start")
button.on_click(get_progress)
display(button)

progress_info = widgets.Output()
info_pane = widgets.VBox([display_area_two],layout=Layout(border="2px solid grey"))
display(info_pane)

Button(description='start', style=ButtonStyle())

## OK, on split, let's try to pass a sub-list into the main control flow! 

In [87]:
# create my iterator and hard code this specific iterator as an argument to a show progress function
spend_iterator = iter(spends)


def get_progress(button):
    """
    this function is basically taking over the function that 
    is being bound to the button, and passing through
    - the button 
    - the iterator that gets us through our list 
    - the data that was turned into an iterator
    """
    show_progress(button, spend_iterator, spends) # hard coding, bad, boo, bad ian. 

tagged_data = []
def show_progress(button, list_iterator, info_list):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, get_progress)) 
    default_buttons = widgets.HBox(set_buttons(["stop", "split"], get_progress)) 
    # the buttons are bound to the get_progress function
    # this basically brings us back into this function, but one step after where we were
    display("hi")
    with display_area_three: 
        current_index = info_list.index(current_value) # use this to find the location in the iter list
        tagged_data = info_list[current_index -1] # look back one to see what we tagged!!

        if button.description == "split":
            clear_output()
            new_iterator = iter(tagged_data)
#             display(button)
            show_progress(button, new_iterator, tagged_data) # try viewing with the new data
            
        else:
            clear_output()
            print(message) 
            display(buttons) 
            # we have a "button" context which was the button that was clicked in the last view
            print(tagged_data)
            print("tagged as: " + button.description)
            display(default_buttons) 

### This ends up entering some crazy kind of a loop

In [73]:
button = widgets.Button(description = "start")
button.on_click(get_progress)
display(button)

display_area_three = widgets.Output()
info_pane = widgets.VBox([display_area_three],layout=Layout(border="2px solid grey"))
display(info_pane)

Button(description='start', style=ButtonStyle())

'hi'

'hi'

# Solve using Partials

## about partials

partials allow us to re-write functions, and implicitly wrap them

In [93]:
# about partials
from functools import partial

tagged_data = []
def callback(button, foo):
    print(foo)

test_partial = widgets.Button(description = "test partial")
test_partial.on_click(partial(callback, foo="passed data"))

layout = widgets.VBox([test_partial])
display(layout)

passed data
passed data
passed data


## Solving our case with Partials

In [150]:
# create my iterator and hard code this specific iterator as an argument to a show progress function


def callback(button, foo):
    tagged_data.append([button.description, foo[0]])
    print(tagged_data)
    show_progress_partial(foo[1])
    
def show_progress_partial(list_iterator):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, partial(callback, foo=(message, list_iterator)))) # partial here decouples the click from the state!
    with display_area_three: 
        clear_output()
        print(message) 
        display(buttons) 
        print(tagged_data)

In [151]:
tagged_data = [] # a list to hold our tagged data in 
spend_iterator = iter(spends) # create our inital iterator 

display_area_three = widgets.Output()

# don't need a button to start as we are not passing state through this function via a button!
show_progress_partial(spend_iterator) 
info_pane = widgets.VBox([display_area_three],layout=Layout(border="2px solid grey"))
display(info_pane)

## Now try the complicated case with control flow

In [36]:
# create my iterator and hard code this specific iterator as an argument to a show progress function

tagged_data = []
def callback(button, foo):
    tagged_data.append([button.description, foo[0]])
    print(tagged_data)
    show_progress_partial(foo[1])
    
def show_progress_partial(list_iterator):
    current_value = next(list_iterator)
    message = current_value
    buttons = widgets.HBox(set_buttons(tags, partial(callback, foo=(message, list_iterator)))) # partial here decouples the click from the state!
#     default_buttons = widgets.HBox(set_buttons(["stop", "split"], get_progress)) 
    with display_area_three: 
        clear_output()
        print(message) 
        display(buttons) 
        print(tagged_data)
        
        

#     # the buttons are bound to the get_progress function
#     # this basically brings us back into this function, but one step after where we were
#     display("hi")
#     with display_area_two: 
#         current_index = info_list.index(current_value) # use this to find the location in the iter list
#         tagged_data = info_list[current_index -1] # look back one to see what we tagged!!

#         if button.description == "split":
#             clear_output()
#             new_iterator = iter(tagged_data)
# #             display(button)
#             show_progress(button, new_iterator, tagged_data) # try viewing with the new data
            
#         else:
#             clear_output()
#             print(message) 
#             display(buttons) 
#             # we have a "button" context which was the button that was clicked in the last view
#             print(tagged_data)
#             print("tagged as: " + button.description)
#             display(default_buttons) 

In [37]:
display_area_three = widgets.Output()

button = widgets.Button(description = "start")
spend_iterator = iter(spends)
button.on_click(show_progress_partial(spend_iterator))

info_pane = widgets.VBox([display_area_three],layout=Layout(border="2px solid grey"))
display(info_pane)

In [38]:
# sketch idea 

# look at the end iteration error 
# if the length of the thing being iterated is 4 
# and we hit the end iterator error 
# then pass back in the longer list 

# if the length of the thing being iterated is longer than 4 
# and we hit the end iterator error 
# then we have finished the data processing 

# Trying to impliment the iPython widget reccomended way of doing this

## using a wrapper for a generator 

In [98]:
# this is a tidier way of doing what we did above, 
# the calling function is not passed explicity to the widget that is wrapped 
#TODO: figure out how to pass data into the button click
from functools import wraps
def yield_for_change(widget):
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.on_click(next_i)
            # start the generator
            next(i)
        return inner
    return f

from ipywidgets import Button
button=Button()

def on_button_clicked():
    print("Button clicked.")

list_to_tag = ["one", "two", "three", "four", "five"]
@yield_for_change(button) # if I need to pass data in, somehow it needs to go in at the binding of the button to the yield
def f():
    for item in list_to_tag: # doing a straight loop
        print('tried to tag ', item)
        x = yield # the loop waits on a yeild statement that is progressed by clicking the button. 

f()
button

tried to tag  one


Button(style=ButtonStyle())

tried to tag  two
tried to tag  three


One of mu big problems with the above code is that I just don't understand it yet

## Using asyncio with a slider

In [99]:
from ipywidgets import IntSlider
import asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

list_to_tag = ["one", "two", "three", "four", "five"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for i in list_to_tag: # again, iterating through the loop 
        print('going to tag ', i)
        x = await wait_for_change(slider, 'value') # using await, and passig a value
        print("tagged ", i, "with  %s"%x)
        print("")

asyncio.ensure_future(f())
slider
#slider

IntSlider(value=0)

tagged  two with  2

going to tag  three
tagged  three with  3

going to tag  four
tagged  four with  4

going to tag  five
tagged  five with  5



In [100]:
## Using the same approach with a button breaks :(

In [101]:
# call function using await - button - 
from ipywidgets import Button

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        print(change)
        future.set_result(change)
        widget.unobserve(getvalue, value)
    widget.on_click(getvalue)
    return future

button=Button(description="wow")

list_to_tag = ["one", "two", "three", "four"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for i in list_to_tag:
        print('going to tag ', i)
        x = await wait_for_change(button, 'value')
        print("tagged ", i, "with  %s"%x)
        print("")
asyncio.create_task(f())

button

Button(description='wow', style=ButtonStyle())

Button(description='wow', style=ButtonStyle())
tagged  one with  Button(description='wow', style=ButtonStyle())

going to tag  two
Button(description='wow', style=ButtonStyle())


InvalidStateError: invalid state

Button(description='wow', style=ButtonStyle())
tagged  two with  Button(description='wow', style=ButtonStyle())

going to tag  three


### Learing through errors

The code blow breaks, but has a really interesting error message, could learn about what is going on here. 
What I think is going on is the widget.observe function is not just accepting a passed in vlaue

In [13]:
from ipywidgets import IntSlider
import asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(value):
        # make the new value available
        future.set_result(value)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

list_to_tag = ["one", "two", "three", "four", "five"]

async def f():
    """
    this function has a weird effect by which the first print is missed, and 
    nothing happens until we interact with the widget, can be gotten around. 
    """
    for item in list_to_tag: # again, iterating through the loop 
        print('going to tag ', item)
        x = await wait_for_change(slider, 'value') # using await, and passig a value
        print("tagged ", item, "with  %s"%x)
        print("")

asyncio.ensure_future(f())
slider
#slider

IntSlider(value=0)

tagged  one with  {'name': 'value', 'old': 0, 'new': 3, 'owner': IntSlider(value=3), 'type': 'change'}

going to tag  two


InvalidStateError: invalid state

InvalidStateError: invalid state

InvalidStateError: invalid state

...

InvalidStateError: invalid state

InvalidStateError: invalid state

InvalidStateError: invalid state

InvalidStateError: invalid state

InvalidStateError: invalid state

# Appendix -  Some basics on binding data to buttons

## binding to a function

In [146]:
# binding to a function
def am_clicked(button):
    print("clicked")

button=Button(description="wow")
button.on_click(am_clicked)
button

Button(description='wow', style=ButtonStyle())

clicked
clicked
clicked
clicked
clicked
clicked
clicked


## binding to a function and passing an argument

This trivial case fails

In [147]:
# binding to a function, and trying to pass an argument
# this calls the function instead of passing the function and argument
def am_clicked(button, arg):
    print(arg)
    print("clicked")

button=Button(description="wow")
arg = "data"
button.on_click(am_clicked, arg)
button

Button(description='wow', style=ButtonStyle())

## binding to a funciton and passing the argument in the function call

This also fails

In [148]:
# binding to a function, and trying to pass an argument
# this calls the function instead of passing the function and argument
def am_clicked(button, arg=arg):
    print(arg)
    print("clicked")

button=Button(description="wow")
arg = "data"
button.on_click(am_clicked(arg))
button

data
clicked


Button(description='wow', style=ButtonStyle())

## using partial to send data to the function bound to the button

In [149]:
# about partials
from functools import partial

tagged_data = []
def am_clicked(button, foo):
    print(foo)

test_partial = widgets.Button(description = "test partial")
test_partial.on_click(partial(am_clicked, foo="passed data"))
test_partial

Button(description='test partial', style=ButtonStyle())

passed data
passed data
passed data
passed data
passed data
passed data
